In [14]:
from pathlib import Path
import speech_recognition as sr
from pydub import AudioSegment


def extract_text_from_audio(audio_file: Path) -> None:
    # convert the MP3 file to a WAV file
    audio_file = str(audio_file)
    if audio_file.endswith(".mp3"):
        wav_file = audio_file.replace(".mp3", ".wav")
        sound = AudioSegment.from_mp3(audio_file)
        sound.export(wav_file, format="wav")
        audio_file = wav_file

    # create a speech recognition object
    r = sr.Recognizer()

    # use the audio file as the source
    with sr.AudioFile(audio_file) as source:
        audio = r.record(source)

    # try to recognize the speech in the audio
    try:
        # text = r.recognize_google(audio)
        text = r.recognize_whisper(audio)
        return text
    except sr.UnknownValueError:
        print("Speech recognition could not understand the audio")
        return None
    except sr.RequestError as e:
        print(
            "Could not request results from Google Speech Recognition service; {0}".format(
                e
            )
        )
        return None

In [25]:
import torch
import whisper
import warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

model = whisper.load_model("base", device)

In [27]:
audio_path = Path("data/audio")

audio_file = audio_path / "eng_ai.mp3"
text = model.transcribe(str(audio_file))['text']
print(text)


audio_file = audio_path / "eng_reg.mp3"
text = model.transcribe(str(audio_file))['text']
print(text)

audio_file = audio_path / "rus_reg.mp3"
text = model.transcribe(str(audio_file))['text']
print(text)

audio_file = audio_path / "rus_ai.mp3"
text = model.transcribe(str(audio_file))['text']
print(text)

 I want to get an internship at MTSC.
 I want to get an internship at MTS.
 Хочу пройти на стажировку ВМТС.
 Хочу пройти на стажировковим TS.
